In [3]:
import torch
import torchvision
import torchvision.datasets as datasets
from tqdm import tqdm
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.backends.cudnn as cudnn
import os
import sys
import logging
import logging.handlers
from PIL import Image

# Parser

In [3]:
parser = argparse.ArgumentParser(description='Domain adaptation')
parser.add_argument("--batch_size", type=int, default="400", help="batch size")
parser.add_argument("--learning_rate", type=float, default=1e-3, help="learning rate")
parser.add_argument("--momentum", type=float, default=0.5, help="momentum")
parser.add_argument("--gpu_num", type=int, default=0, help="gpu num")
parser.add_argument("--seed", type=int, default=123, help="munually set seed")
parser.add_argument("--save_path", type=str, default="../train_related", help="save path")
parser.add_argument("--subfolder", type=str, default='domain_origin_dann_svhn_to_mnist', help="subfolder name")
parser.add_argument("--wtarget", type=float, default=0.7, help="target weight")
parser.add_argument("--model_save_period", type=int, default=2, help="save period")
parser.add_argument("--epochs", type=int, default=2000, help="label shuffling")
parser.add_argument("--dann_weight", type=float, default=1, help="weight for label shuffling")
parser.add_argument("--start_origin_dann", type=int, default=100, help="when to start shuffling")
parser.add_argument("--use_gpu", type=int, default=0, help="whether using gpu to train")


args = parser.parse_args()
# snap shot of py file and command
python_file_name = sys.argv[0]


usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH] [--task TASK]
                             [--gpu_num GPU_NUM] [--batch_size BATCH_SIZE]
                             [--epochs EPOCHS] [--lr_FNN LR_FNN]
                             [--lr_encoder LR_ENCODER]
                             [--lr_centerloss LR_CENTERLOSS] [--sclass SCLASS]
                             [--scent SCENT] [--lr_gan LR_GAN]
                             [--target_lbl_percentage TARGET_LBL_PERCENTAGE]
                             [--source_lbl_percentage SOURCE_LBL_PERCENTAGE]
                             [--num_per_class NUM_PER_CLASS] [--seed SEED]
                             [--save_path SAVE_PATH]
                             [--model_save_period MODEL_SAVE_PERIOD]
                             [--epoch_begin_prototype EPOCH_BEGIN_PROTOTYPE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Tianqin Li\AppData\Roaming\jupyter\runtime\kernel-460988ff-287e-4235-b5eb-83de570d2508.json


SystemExit: 2

C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
# # local only
# class local_args:
#     def __init__(self, **entries):
#         self.__dict__.update(entries)
        
# args = local_args(**{
#     'batch_size': 400,
#     'learning_rate': 1e-3,
#     'momentum': 0.5,
#     'gpu_num': 0,
#     'seed': 123,
#     'save_path': "../train_related",
#     'epochs': 2,
#     'subfolder': "testing",
#     'wtarget': 0.7,
#     'dann_weight': 1,
#     'model_save_period': 2,
#     'start_origin_dann': 0,
#     'use_gpu': 0,
# })

In [7]:
if args.use_gpu == 1 and torch.cuda.is_available():
    device = torch.device('cuda:{}'.format(args.gpu_num))
else:
    device = 'cpu'
print(device)
# seed
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)
cudnn.deterministic = True
torch.backends.cudnn.deterministic = True






model_sub_folder = args.subfolder + '/origin_dann_weight_%f_learningrate_%f'%(args.dann_weight, args.learning_rate)
save_folder = os.path.join(args.save_path, model_sub_folder)
if not os.path.exists(save_folder):
    os.makedirs(save_folder)   


# Logger

In [8]:

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logfile_path = os.path.join(save_folder, 'logfile.log')
if os.path.isfile(logfile_path):
    os.remove(logfile_path)
    
file_log_handler = logging.FileHandler(logfile_path)
logger.addHandler(file_log_handler)
logger.info("Fixed source testing bug")
stdout_log_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stdout_log_handler)

attrs = vars(args)
for item in attrs.items():
    logger.info("%s: %s"%item)
logger.info("Training Save Path: {}".format(save_folder))


data_path: ../data_unzip
task: 3E
num_class: 50
batch_size: 100
num_per_class: 2
gap: 5
lbl_percentage: 0.7
lr_gan: 0.0001
lr_FNN: 0.0001
lr_encoder: 0.0001
epochs: 2
clip_value: 0.01
n_critic: 4
sclass: 0.7
scent: 0.01
seed: 0
save_path: ../train_related
model_save_period: 1
lr_centerloss: 0.001
lr_prototype: 0.001
sprototype: 0.01
select_pretrain_epoch: 77
epoch_begin_prototype: 0
sbinary_loss: 1
gpu_num: 0
source_lbl_percentage: 0.7
target_lbl_percentage: 0.7


# Data Loading


In [9]:
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
svhn_trainset = datasets.SVHN(root='../data', split='train', download=True, transform=torchvision.transforms.Compose([
                    torchvision.transforms.Resize((28, 28)),
                    torchvision.transforms.Grayscale(num_output_channels=1),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize([0.5], [0.5])]))
svhn_testset = datasets.SVHN(root='../data', split='test', download=True, transform=torchvision.transforms.Compose([
                    torchvision.transforms.Resize((28, 28)),
                    torchvision.transforms.Grayscale(num_output_channels=1),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize([0.5], [0.5])]))

In [ ]:
# reload data
train_mnist_loader = DataLoader(mnist_trainset, batch_size=args.batch_size, shuffle=True)
test_mnist_loader = DataLoader(mnist_testset, batch_size=args.batch_size, shuffle=True)
train_svhn_loader = DataLoader(svhn_trainset, batch_size=args.batch_size, shuffle=True)
test_svhn_loader = DataLoader(svhn_testset, batch_size=args.batch_size, shuffle=True)

# Weight initialize

In [10]:
def weights_init(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
    elif type(m) == nn.LayerNorm:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)


# Model creation

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 30)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x


class FNN(nn.Module):
    def __init__(self, d_in, d_h1, d_h2, d_out, dp=0.2):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(d_in, d_h1)
        self.ln1 = nn.LayerNorm(d_h1)
        self.relu1 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(dp)
        self.fc2 = nn.Linear(d_h1, d_h2)
        self.ln2 = nn.LayerNorm(d_h2)
        self.relu2 = nn.ReLU(inplace=True)
        self.dropout2 = nn.Dropout(dp)
        self.fc3 = nn.Linear(d_h2, d_out)

    def forward(self, x):
        x = self.fc1(x)
        x = self.ln1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.ln2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x
    
    def before_lastlinear(self, x):
        x = self.fc1(x)
        x = self.ln1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.ln2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        return x

        

In [ ]:
class Gfunction(nn.Sequential):
    def __init__(self):
        super(Gfunction, self).__init__(
            nn.Linear(200,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,1)
        )

In [1]:
def log_mean_exp(x, device):
    max_score = x.max()
    batch_size = torch.Tensor([x.shape[0]]).to(device)
    stable_x = x - max_score
    return max_score - batch_size.log() + stable_x.exp().sum(dim=0).log()

a = torch.rand([100,1]).to(device)
assert torch.all(log_mean_exp(a, device) - a.exp().mean(dim=0).log() < 1e-6)

NameError: name 'torch' is not defined

In [276]:
def KLDiv(g_x_source, g_x_target, device):
    # clipping
#     g_x_source = torch.clamp(g_x_source, -1e3, 1e3)
#     g_x_target = torch.clamp(g_x_target, -1e3, 1e3)
    return g_x_source.mean(dim=0) - log_mean_exp(g_x_target, device)

In [ ]:
def JSDiv(g_x_source, g_x_target, device):
    return -F.softplus(-g_x_source).mean(dim=0) - F.softplus(g_x_target).mean(dim=0)

In [52]:

torch.manual_seed(args.seed)
if device == 'cuda:0': 
    torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)

encoder = Encoder().to(device)
CNet = FNN(d_in=30, d_h1=100, d_h2=100, d_out=10, dp=0.2).to(device)
DomainCNet = FNN(d_in=30, d_h1=100, d_h2=100, d_out=2, dp=0.2).to(device)




optimizerEncoder = optim.Adam(encoder.parameters(), lr=args.learning_rate)
optimizerCNet = optim.Adam(CNet.parameters(), lr=args.learning_rate)
optimizerDomainCNet = optim.Adam(DomainCNet.parameters(), lr=args.learning_rate)

criterion_classifier = nn.CrossEntropyLoss().to(device)
# criterion_adverisal = 

encoder.apply(weights_init)
CNet.apply(weights_init)
DomainCNet.apply(weights_init)






if args.KL:
    gfunction_KL_div_labeled = Gfunction().to(device)
    gfunction_KL_div_unlabeled = Gfunction().to(device)

if args.JS:
    gfunction_JS_div_labeled = Gfunction().to(device)
    gfunction_JS_div_unlabeled = Gfunction().to(device)
    
if args.classifier:
    CNet = SimpleMLP2().to(device)
    criterion_classifier = nn.CrossEntropyLoss().to(device)


cpu


NameError: name 'num_class' is not defined

# Train

In [16]:
logger.info('Started loading')
source_acc_label_ = np.load(os.path.join(args.model_path, 'source_acc_label_.npy'))
source_acc_unlabel_ = np.load(os.path.join(args.model_path, 'source_acc_unlabel_.npy'))
target_acc_label_ = np.load(os.path.join(args.model_path, 'target_acc_label_.npy'))
target_acc_unlabel_ = np.load(os.path.join(args.model_path, 'target_acc_unlabel_.npy'))

labeled_KL = []
unlabeled_KL = []
labeled_JS = []
unlabeled_JS = []
acc_source_unlabeled_classifier_ = []
acc_target_unlabeled_classifier_ = []

source_acc_label = []
source_acc_unlabel = []
target_acc_label = []
target_acc_unlabel = []

epochs = []

for epoch in range(3, source_acc_label_.shape[0], args.intervals*args.model_save_period):
    # initialize 
    if args.KL:
        gfunction_KL_div_labeled.apply(weights_init)
        optimizer_gfunction_KL_div_labeled = torch.optim.Adam(gfunction_KL_div_labeled.parameters(), lr=args.lr)
        gfunction_KL_div_unlabeled.apply(weights_init)
        optimizer_gfunction_KL_div_unlabeled = torch.optim.Adam(gfunction_KL_div_unlabeled.parameters(), lr=args.lr)

    if args.JS:
        gfunction_JS_div_labeled.apply(weights_init)
        optimizer_gfunction_JS_div_labeled = torch.optim.Adam(gfunction_JS_div_labeled.parameters(), lr=args.lr)
        gfunction_JS_div_unlabeled.apply(weights_init)
        optimizer_gfunction_JS_div_unlabeled = torch.optim.Adam(gfunction_JS_div_unlabeled.parameters(), lr=args.lr)

    if args.classifier:
        CNet.load_state_dict(torch.load(os.path.join(args.model_path, 'CNet_%i.t7'%epoch)))
        optimizer_CNet = torch.optim.Adam(CNet.parameters(), lr=args.lr)
    
    # load weight
    encoder.load_state_dict(torch.load(os.path.join(args.model_path, 'encoder_%i.t7'%epoch)))
    
    # inferencing
    encoder.eval()
    
    # get source/target embedding
    source_x_labeled_embedding = torch.empty(0).to(device)
    source_y_labeled = torch.empty(0).long().to(device)
    source_x_unlabeled_embedding = torch.empty(0).to(device)
    source_y_unlabeled = torch.empty(0).long().to(device)
    target_x_labeled_embedding = torch.empty(0).to(device)
    target_y_labeled = torch.empty(0).long().to(device)
    target_x_unlabeled_embedding = torch.empty(0).to(device)
    target_y_unlabeled = torch.empty(0).long().to(device)
    with torch.no_grad():
        for batch_id, (source_x, source_y) in tqdm(enumerate(labeled_source_dataloader), total=len(labeled_source_dataloader)):
            source_x = source_x.to(device).view(-1,3200).float()
            source_y = source_y.to(device).long()
            source_x_embedding = encoder(source_x).detach()
            source_x_labeled_embedding = torch.cat([source_x_labeled_embedding, source_x_embedding])
            source_y_labeled = torch.cat([source_y_labeled, source_y])
            
        for batch_id, (source_x, source_y) in tqdm(enumerate(unlabeled_source_dataloader), total=len(unlabeled_source_dataloader)):
            source_x = source_x.to(device).view(-1,3200).float()
            source_y = source_y.to(device).long()
            source_x_embedding = encoder(source_x).detach()
            source_x_unlabeled_embedding = torch.cat([source_x_unlabeled_embedding, source_x_embedding])
            source_y_unlabeled = torch.cat([source_y_unlabeled, source_y])
            
        for batch_id, (target_x, target_y) in tqdm(enumerate(labeled_target_dataloader), total=len(labeled_target_dataloader)):
            target_x = target_x.to(device).view(-1,3200).float()
            target_y = target_y.to(device).long()
            fake_x_embedding = encoder(target_x).detach()
            target_x_labeled_embedding = torch.cat([target_x_labeled_embedding, fake_x_embedding])     
            target_y_labeled = torch.cat([target_y_labeled, target_y])

            
        for batch_id, (target_x, target_y) in tqdm(enumerate(unlabeled_target_dataloader), total=len(unlabeled_target_dataloader)):
            target_x = target_x.to(device).view(-1,3200).float()
            target_y = target_y.to(device).long()
            fake_x_embedding = encoder(target_x).detach()
            target_x_unlabeled_embedding = torch.cat([target_x_unlabeled_embedding, fake_x_embedding])    
            target_y_unlabeled = torch.cat([target_y_unlabeled, target_y])
            
    # for loop to train the gfunction 
    for i in tqdm(range(args.gfunction_epoch)):
        if args.KL:
            optimizer_gfunction_KL_div_labeled.zero_grad()
            source_x_labeled_g = gfunction_KL_div_labeled(source_x_labeled_embedding)
            target_x_labeled_g = gfunction_KL_div_labeled(target_x_labeled_embedding)
            loss_KL_labeled = - KLDiv(source_x_labeled_g, target_x_labeled_g, device) # maximize
            loss_KL_labeled.backward()
            optimizer_gfunction_KL_div_labeled.step()
            if i % 500 == 0:
                print("Epoch %i, Iter %i, labeled KL: %f"%(epoch, i, -loss_KL_labeled.item()))
       
        if args.JS:
            optimizer_gfunction_JS_div_labeled.zero_grad()
            source_x_labeled_g = gfunction_JS_div_labeled(source_x_labeled_embedding)
            target_x_labeled_g = gfunction_JS_div_labeled(target_x_labeled_embedding)
            loss_JS_labeled = - JSDiv(source_x_labeled_g, target_x_labeled_g, device) # maximize
            loss_JS_labeled.backward()
            optimizer_gfunction_JS_div_labeled.step()
            if i % 500 == 0:
                print("Epoch %i, Iter %i, labeled JS: %f"%(epoch, i, -loss_JS_labeled.item()))
            
    if args.KL:
        loss_KL_labeled = - loss_KL_labeled.item()
        labeled_KL.append(loss_KL_labeled)
      
    if args.JS:
        loss_JS_labeled = - loss_JS_labeled.item()
        labeled_JS.append(loss_JS_labeled)
    
    for i in tqdm(range(args.gfunction_epoch)):
        if args.KL:
            optimizer_gfunction_KL_div_unlabeled.zero_grad()
            source_x_unlabeled_g = gfunction_KL_div_unlabeled(source_x_unlabeled_embedding)
            target_x_unlabeled_g = gfunction_KL_div_unlabeled(target_x_unlabeled_embedding)
            loss_KL_unlabeled = - KLDiv(source_x_unlabeled_g, target_x_unlabeled_g, device) # maximize
            loss_KL_unlabeled.backward()
            optimizer_gfunction_KL_div_unlabeled.step()
            if i % 500 == 0:
                print("Epoch %i, Iter %i, unlabeled KL: %f"%(epoch, i, -loss_KL_unlabeled.item()))

        if args.JS:
            optimizer_gfunction_JS_div_unlabeled.zero_grad()
            source_x_unlabeled_g = gfunction_JS_div_unlabeled(source_x_unlabeled_embedding)
            target_x_unlabeled_g = gfunction_JS_div_unlabeled(target_x_unlabeled_embedding)
            loss_JS_unlabeled = - JSDiv(source_x_unlabeled_g, target_x_unlabeled_g, device) # maximize
            loss_JS_unlabeled.backward()
            optimizer_gfunction_JS_div_unlabeled.step()
            if i % 500 == 0:
                print("Epoch %i, Iter %i, unlabeled JS: %f"%(epoch, i, -loss_JS_unlabeled.item()))
            
    if args.KL:  
        loss_KL_unlabeled = - loss_KL_unlabeled.item()
        unlabeled_KL.append(loss_KL_unlabeled)
    
    if args.JS:
        loss_JS_unlabeled = - loss_JS_unlabeled.item()
        unlabeled_JS.append(loss_JS_unlabeled)

    acc_source_labeled_classifier = 0
    acc_target_labeled_classifier = 0
    if args.classifier:
#         while i < args.classifier_epoch or (acc_source_labeled_classifier < 0.98 and acc_target_labeled_classifier < 0.98):
#             i += 1
        for i in tqdm(range(args.classifier_epoch)):
            CNet.train()
            optimizer_CNet.zero_grad()
            pred = CNet(source_x_labeled_embedding)
            acc_source_labeled_classifier = (pred.argmax(-1) == source_y_labeled).sum().item() / pred.size(0)
            loss_source_classifier_labeled = criterion_classifier(pred, source_y_labeled) * args.sclass
            if args.centerloss: loss_source_classifier_labeled += criterion_centerloss(source_x_labeled_embedding, source_y_labeled) * args.scent * args.sclass
            loss_source_classifier_labeled.backward()
            if args.centerloss: optimizer_centerloss.step()
            optimizer_CNet.step()
            
            optimizer_CNet.zero_grad()
            pred = CNet(target_x_labeled_embedding)
            acc_target_labeled_classifier = (pred.argmax(-1) == target_y_labeled).sum().item() / pred.size(0)
            loss_target_classifier_labeled = criterion_classifier(pred, target_y_labeled)
            loss_target_classifier_labeled.backward()
            optimizer_CNet.step()
            
#             if i % 500 == 0:
#                 CNet.eval()
#                 pred = CNet(source_x_unlabeled_embedding)
#                 acc_source_unlabeled_classifier = (pred.argmax(-1) == source_y_unlabeled).sum().item() / pred.size(0)
#                 pred = CNet(target_x_unlabeled_embedding)
#                 acc_target_unlabeled_classifier = (pred.argmax(-1) == target_y_unlabeled).sum().item() / pred.size(0)
#                 print("Iter %i: source acc: labeled: %f, unlabeled: %f; target acc: labeled: %f, unlabeled: %f"%(
#                     i, acc_source_labeled_classifier, acc_source_unlabeled_classifier, acc_target_labeled_classifier, acc_target_unlabeled_classifier))
        
        CNet.eval()
        pred = CNet(source_x_unlabeled_embedding)
        acc_source_unlabeled_classifier = (pred.argmax(-1) == source_y_unlabeled).sum().item() / pred.size(0)
        pred = CNet(target_x_unlabeled_embedding)
        acc_target_unlabeled_classifier = (pred.argmax(-1) == target_y_unlabeled).sum().item() / pred.size(0)
        acc_source_unlabeled_classifier_.append(acc_source_unlabeled_classifier)
        acc_target_unlabeled_classifier_.append(acc_target_unlabeled_classifier)
        
    # save corresponding acc
    source_acc_label.append(source_acc_label_[epoch-1])
    source_acc_unlabel.append(source_acc_unlabel_[epoch-1])
    target_acc_label.append(target_acc_label_[epoch-1])
    target_acc_unlabel.append(target_acc_unlabel_[epoch-1])
    epochs.append(epoch)
    
    logger.info("-----------------------------------------")
    log_string = "Epoch %i: "%epoch
    if args.KL: log_string += "labeled KL: %f, unlabeled KL: %f; "%(loss_KL_labeled, loss_KL_unlabeled)
    if args.JS: log_string += "labeled JS: %f, unlabeled JS: %f; "%(loss_JS_labeled, loss_JS_unlabeled)   
    if args.classifier: log_string += "src unlbl acc: %f, tgt unlbl acc: %f; "%(acc_source_unlabeled_classifier, acc_target_unlabeled_classifier)      
    logger.info(log_string)
    logger.info("-----------------------------------------")
    
    np.save(args.save_path+model_sub_folder+'/epochs.npy', epochs)
    np.save(args.save_path+model_sub_folder+'/source_acc_label.npy', source_acc_label)
    np.save(args.save_path+model_sub_folder+'/source_acc_unlabel.npy', source_acc_unlabel)
    np.save(args.save_path+model_sub_folder+'/target_acc_label.npy', target_acc_label)
    np.save(args.save_path+model_sub_folder+'/target_acc_unlabel.npy', target_acc_unlabel)
    
    if args.KL: 
        np.save(args.save_path+model_sub_folder+'/labeled_KL.npy', labeled_KL)
        np.save(args.save_path+model_sub_folder+'/unlabeled_KL.npy', unlabeled_KL)
        
    if args.JS:
        np.save(args.save_path+model_sub_folder+'/labeled_JS.npy', labeled_JS)
        np.save(args.save_path+model_sub_folder+'/unlabeled_JS.npy', unlabeled_JS)
        
    if args.classifier:
        np.save(args.save_path+model_sub_folder+'/acc_source_unlabeled_classifier_.npy', acc_source_unlabeled_classifier_)
        np.save(args.save_path+model_sub_folder+'/acc_target_unlabeled_classifier_.npy', acc_target_unlabeled_classifier_)
    

Started Training


  0%|          | 0/99 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 31, in _pin_memory_loop
    data = pin_memory(data)
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 55, in pin_memory
    return [pin_memory(sample) for sample in data]
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 55, in <listcomp>
    return [pin_memory(sample) for sample in data]
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 47, in pin_memory
    return data.pin_memory()
RuntimeError: cuda runtime error (710) : device-side assert triggered at ..\aten\src\THC\THCCachingHostAllocator.cpp:278
